In [1]:

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.context import SQLContext
from pyspark.sql.functions import udf
from pyspark.sql.functions import lit
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.sql.functions import lit, udf
import pyspark.sql.types as T 
import argparse
from configparser import ConfigParser
import json

conf = SparkConf()
conf.setAppName('pillar')
# sc = SparkContext(conf=conf)
sc = SparkContext.getOrCreate(conf = conf);
spark = SparkSession(sc)
sc.setLogLevel('WARN')
sql_context = SQLContext(sc)
spark = SparkSession.builder.appName('pillar').getOrCreate()

In [2]:
df = spark.read.parquet("input.parquet/*")

In [4]:
# op = spark.read.parquet("output.parquet/*")
# op.toPandas()

,item_name,age,gender,weighted_sum
0,Pop Goes the Weasel2,2,female,1.66


In [ ]:
expected = pd.DataFrame({'item_name':['1','1','1','2','2','2','3','3','3','4','4','4','5','5','5','6','6','6'],
			'age':['M','M','M','M','M','M','M','M','M','F','F','F','F','F','F','F','F','F'],
			'gender':['active','rare','inactive','active','rare','inactive','active','rare','inactive','active','rare','inactive','active','rare','inactive','active','rare','inactive'],
			'weighted_sum':['1','2','3','1','2','3','1','2','3','1','2','3','1','2','3','1','2','3'])

In [9]:
df.toPandas()

,device_id,item_name,PercentPlayed,weight
0,1,Pop Goes the Weasel2,2.0,1.00
1,1,Pop Goes the Weasel2,0.5,0.66


In [6]:
import pandas as pd
profile_pd = pd.DataFrame({'device_id':['00','01','02','03','04','05'],
			'gender':['M','M','M','F','F','M'],
			'age':['1','2','3','4','1','1']})
profile_pd

,age,device_id,gender
0,1,00,M
1,2,01,M
2,3,02,M
3,4,03,F
4,5,04,F
5,6,05,F


In [13]:
expected = pd.DataFrame({'item_name':['Five Little Monkeys','Humpty Dumpty','Humpty Dumpty','Humpty Dumpty','Humpty Dumpty','Pop Goes the Weasel2', 'Pop Goes the Weasel2', 'Pop Goes the Weasel2', 'Pop Goes the Weasel2', 'Pop Goes the Weasel2', 'The Wheels on the Bus'],
			'age':[1, 1, 1, 2, 3, 1, 1, 2, 3, 4, 4],
			'gender':['M','F','M','M','M','F','M','M','M','F','F'],
			'weighted_sum':[0.33, 1.0, 2.0, 0.33, 1.0, 0.0, 0.33, 0.66, 1.0, 1.65, 1.0]})
expected

,age,gender,item_name,weighted_sum
0,1,M,Five Little Monkeys,0.33
1,1,F,Humpty Dumpty,1.00
2,1,M,Humpty Dumpty,2.00
3,2,M,Humpty Dumpty,0.33
4,3,M,Humpty Dumpty,1.00
5,1,F,Pop Goes the Weasel2,0.00
6,1,M,Pop Goes the Weasel2,0.33
7,2,M,Pop Goes the Weasel2,0.66
8,3,M,Pop Goes the Weasel2,1.00
9,4,F,Pop Goes the Weasel2,1.65


In [10]:
df_pd = pd.DataFrame({'device_id':['00','00','00','01','01','01','02','02','02','03','03','03','04','04','04','05','05','05'],
			'item_name':['Pop Goes the Weasel2','Pop Goes the Weasel2','Humpty Dumpty','Pop Goes the Weasel2','Humpty Dumpty','Pop Goes the Weasel2','Humpty Dumpty','Humpty Dumpty','Pop Goes the Weasel2','Pop Goes the Weasel2','Pop Goes the Weasel2','Pop Goes the Weasel2','Humpty Dumpty','Pop Goes the Weasel2','The Wheels on the Bus','Five Little Monkeys','Humpty Dumpty','Humpty Dumpty'],
			'PercentPlayed':[0.3,0.1,1.0,0.6,0.5,0.2,1.0,0.1,1.0,0.3,0.75,0.7,0.9,0.1,1.0,0.3,0.1,1.0],
            'weight':[0.33,0.0,1.0,0.66,0.33,0.0,1.0,0.0,1.0,0.33,0.66,0.66,1.0,0.0,1.0,0.33,0.0,1.0]
			})
df_pd

,PercentPlayed,device_id,item_name,weight
0,0.30,00,Pop Goes the Weasel2,0.33
1,0.10,00,Pop Goes the Weasel2,0.00
2,1.00,00,Humpty Dumpty,1.00
3,0.60,01,Pop Goes the Weasel2,0.66
4,0.50,01,Humpty Dumpty,0.33
5,0.20,01,Pop Goes the Weasel2,0.00
6,1.00,02,Humpty Dumpty,1.00
7,0.10,02,Humpty Dumpty,0.00
8,1.00,02,Pop Goes the Weasel2,1.00
9,0.30,03,Pop Goes the Weasel2,0.33


In [10]:
genderage_df = spark.read.csv('profile.csv', header=True)
df = df.join(genderage_df, 'device_id', 'inner')

df = df.select('item_name', 'weight', 'age', 'gender')
df = df.groupBy(['item_name', 'age', 'gender']).agg(df.sum("weight").alias("weighted_sum"))
df.show()

AttributeError: 'DataFrame' object has no attribute 'sum'

In [5]:
def main(inputDir,profileMap,outputDir,config):
    profile = spark.read.csv(profileMap, header='true')
    df = spark.read.parquet(inputDir+"/*")
    df = df.join(profile, 'device_id')
    df = df.select('item_name', 'weight', 'age', 'gender')
    df = df.groupBy(['item_name', 'age', 'gender']).sum().alias('weighted_sum')
    df.write.parquet(outputDir)

In [6]:
if __name__ == "__main__":
    parser = argparse.ArgumentParser() 
    parser.add_argument('-i', '--input', required=True)
    parser.add_argument('-m', '--map', required=True)
    parser.add_argument('-o', '--output', required=True)
    parser.add_argument('-c', '--config', required=True)
    args = parser.parse_args()
    config = ConfigParser()
    config.read(args.config)
    main(args.input,args.map,args.output,config)

usage: ipykernel_launcher.py [-h] -i INPUT -m MAP -o OUTPUT -c CONFIG
ipykernel_launcher.py: error: the following arguments are required: -i/--input, -m/--map, -o/--output, -c/--config


SystemExit: 2

/Users/elaine/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
